In [2]:
import numpy as np
import pandas as pd
import torch

In [4]:
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [10]:
df=pd.read_csv('toxic_comment_classification.csv')

In [11]:
df.head()

,text,label
0,sockpuppet how am i a sockpuppet of this so c...,0
1,balls courage this could definitely be a wiki...,0
2,one of the most important parts of this assign...,0
3,"first of all, no matter how much women say the...",1
4,Gay people are equally capable of hate. Ask my...,1


In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

In [9]:
#model=model.to('cuda')

In [29]:
#Spliting data:
X=df['text'].tolist()
y=df['label'].tolist()

In [ ]:
X

In [ ]:
y

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val=train_test_split(X,y, test_size=0.20, stratify=y)

In [31]:
X_train_tokenized=tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized=tokenizer(X_val, padding=True, truncation=True, max_length=512)
#y_train_tokenized=tokenizer(y_train, padding=True, truncation=True, max_length=512)

In [32]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [33]:
#Fine tuning BERT model
#Creating torch custom dataset

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [34]:
train_dataset=Dataset(X_train_tokenized, y_train)
val_dataset=Dataset(X_val_tokenized, y_val)

In [ ]:
train_dataset[5]

In [40]:
#Defining computing metrics function

In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [42]:
def compute_metrics(p):
    print(type(p))
    pred=np.argmax(pred, axis=1)
    accuracy=accuracy_score(y_true=labels, y_pred=pred)
    recall=recall_score(y_true=labels, y_pred=pred)
    precision=precision_score(y_true=labels, y_pred=pred)
    f1=f1_score(y_true=labels, y_pred=pred)

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

In [49]:
#Define Trainer
args= TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`